# Visualize Commute Routes
This notebook visualizes the commute routes generated by `main.py`. The routes are loaded from the `commute_routes.csv` file and plotted on an interactive map using Plotly.

In [48]:
import gcsfs
import pandas as pd
import plotly.express as px
import pytz

In [49]:
# Set your project ID and bucket name
project_id = 'dig-es-nws-gemini-projects'
bucket_name = 'marketplace-commutes'

# Initialize gcsfs
gcs = gcsfs.GCSFileSystem(project=project_id)

# List all CSV files in the bucket
files = gcs.glob(f'gs://{bucket_name}/*.csv')

print(files)

# Read all CSV files into a list of DataFrames
all_dfs = []
for f in files:
    df_temp = pd.read_csv(f"gs://{f}")
    # Extract timezone from filename
    filename = f.split('/')[-1]
    if 'atlantic' in filename:
        df_temp['timezone'] = 'America/Halifax'
    elif 'central' in filename:
        df_temp['timezone'] = 'America/Winnipeg'
    elif 'eastern' in filename:
        df_temp['timezone'] = 'America/Toronto'
    elif 'mountain' in filename:
        df_temp['timezone'] = 'America/Edmonton'
    elif 'pacific' in filename:
        df_temp['timezone'] = 'America/Vancouver'
    else:
        df_temp['timezone'] = 'UTC' # Default or handle as needed
    all_dfs.append(df_temp)


# Concatenate all DataFrames into a single DataFrame
df = pd.concat(all_dfs, ignore_index=True)

# Display the first few rows of the combined DataFrame
df.head()

['marketplace-commutes/commute_routes_atlantic.csv', 'marketplace-commutes/commute_routes_central.csv', 'marketplace-commutes/commute_routes_eastern.csv', 'marketplace-commutes/commute_routes_mountain.csv', 'marketplace-commutes/commute_routes_pacific.csv']


,origin,destination,travel_mode,distance_km,duration_min,warnings,line_geometry,timestamp,timezone
0,"Larry Uteck, Halifax",Hospital in Halifax,DRIVE,10.113,20,NaN,"LINESTRING (-63.67528 44.70207, -63.67451 44.7...",2025-08-26 20:10:05,America/Halifax
1,"Windsor Street, Halifax","St. Margaret's Bay Road, Halifax",DRIVE,5.678,13,NaN,"LINESTRING (-63.60553 44.65555, -63.60527 44.6...",2025-08-26 20:10:05,America/Halifax
2,"Armdale, Halifax","Bayers Lake, Halifax",DRIVE,7.775,12,NaN,"LINESTRING (-63.60788 44.63002, -63.60795 44.6...",2025-08-26 20:10:06,America/Halifax
3,"North End, Halifax","Macdonald Bridge, Halifax",BICYCLE,1.723,10,NaN,"LINESTRING (-63.60227 44.65962, -63.60234 44.6...",2025-08-26 20:10:06,America/Halifax
4,"Larry Uteck, Halifax",Hospital in Halifax,DRIVE,10.113,21,NaN,"LINESTRING (-63.67528 44.70207, -63.67451 44.7...",2025-08-26 20:20:05,America/Halifax


In [50]:
df["timezone"].unique()

array(['America/Halifax', 'America/Winnipeg', 'America/Toronto',
       'America/Edmonton', 'America/Vancouver'], dtype=object)

In [51]:
# --- Data Preparation ---

# Convert timestamp to datetime objects
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)

# Create a unique identifier for each route
df['route_id'] = df['origin'] + ' to ' + df['destination']

# Localize timestamp and extract hour
df['timestamp_local'] = df.apply(lambda row: row['timestamp'].tz_convert(row['timezone']), axis=1)

df['hour'] = df['timestamp_local'].apply(lambda x: x.hour)

# --- Visualization ---
fig = px.line(df, x='timestamp_local', y='duration_min', color='route_id',
              title='Commute Time Over Time',
              labels={'timestamp': 'Timestamp', 'duration_min': 'Commute Time (minutes)'})

fig.show()

In [ ]:
# Filter for off-peak hours
off_peak_df = df[~((df['hour'] >= 7) & (df['hour'] < 9) | (df['hour'] >= 17) & (df['hour'] < 19))]

# Calculate the median travel time for each route during off-peak hours
off_peak_stats = off_peak_df.groupby('route_id')['duration_min'].median().reset_index()
off_peak_stats.columns = ['route_id', 'median_off_peak_duration_min']

# Filter for peak hours
peak_df = df[((df['hour'] >= 7) & (df['hour'] < 9) | (df['hour'] >= 17) & (df['hour'] < 19))]

# Calculate the median and 95th percentile travel time for each route during peak hours
peak_stats = peak_df.groupby('route_id')['duration_min'].agg(['median', lambda x: x.quantile(0.95)]).reset_index()
peak_stats.columns = ['route_id', 'median_peak_duration_min', 'p95_peak_duration_min']

# Merge the stats
final_stats = pd.merge(off_peak_stats, peak_stats, on='route_id', how='left')

# Add calculated columns
final_stats['peak_vs_off_peak_median_diff'] = final_stats['median_peak_duration_min'] - final_stats['median_off_peak_duration_min']
final_stats['p95_vs_median_peak_diff'] = final_stats['p95_peak_duration_min'] - final_stats['median_peak_duration_min']

# Calculate the score and sort by it
final_stats['score'] = (final_stats['peak_vs_off_peak_median_diff'] + final_stats['p95_vs_median_peak_diff']) / final_stats['median_off_peak_duration_min']
final_stats = final_stats.sort_values(by='score', ascending=False)


print("Travel time statistics:")
display(final_stats)

Travel time statistics:


,route_id,median_off_peak_duration_min,median_peak_duration_min,p95_peak_duration_min,peak_vs_off_peak_median_diff,p95_vs_median_peak_diff,score
4,"Cloverdale, Surrey to Richmond/Vancouver",53.5,64.0,71.85,10.5,7.85,18.35
0,"88th Ave, Surrey to Coquitlam",33.0,39.0,43.85,6.0,4.85,10.85
18,"St. James, Winnipeg to Transcona, Winnipeg",27.0,28.0,36.55,1.0,8.55,9.55
3,"Charleswood, Winnipeg to St. James, Winnipeg",17.5,21.0,25.00,3.5,4.00,7.50
15,"Orleans, Ottawa to Downtown Ottawa",20.0,21.5,25.85,1.5,4.35,5.85
19,"St. Norbert, Winnipeg to Polo Park, Winnipeg",79.0,81.0,84.85,2.0,3.85,5.85
10,"Gardiner Expressway and Highway 427, Toronto t...",19.0,22.0,24.00,3.0,2.00,5.00
11,"Larry Uteck, Halifax to Hospital in Halifax",16.0,18.0,21.00,2.0,3.00,5.00
25,"Windsor Street, Halifax to St. Margaret's Bay ...",9.0,9.0,13.90,0.0,4.90,4.90
17,"Riverside South, Ottawa to Downtown Ottawa",26.0,28.0,30.85,2.0,2.85,4.85
